<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_6_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Hugging Face Serverless Inference API
---
På **Hugging Face** finns många modeller. Vi accessar modellerna via ändpunkter - ingen nedladdning av modellerna till Colab.
Det verkar som om vissa modeller inte alltid är laddade till ändpunkten. Om du får ett felmeddelande eller konstiga fel vänta en stund (20 sek) och kör om cellen där API:et används.

In [ ]:
import matplotlib.pyplot as plt
import requests
from PIL import Image
from matplotlib.patches import Rectangle
import numpy as np

In [ ]:
# hämta fil via länk och spara som lokal fil
def get_img_to_file(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
      handler.write(img_data)

In [ ]:
# Access key till Hugging Face
# Du måste skapa en egen nyckel:
# 1) Gå till huggingface.co
# 2) Skapa ett HF-konto (sign in)
# 3) Logga in
# 4) Klicka på användar-ikonen (överst till höger) och välj 'Access Tokens'
# 5) Skapa en ny token (read) och kopiera textsträngen
# 6) Klicka på nyckeln till vänster i Colab
# 7) Klicka på 'Lägg till ny hemlighet'
# 8) Skriv HF_KEY i vänster fält och kopiera in textsträngen i höger

from google.colab import userdata
HF_KEY = userdata.get('HF_KEY')

---
#TEXT-TO-IMAGE
---
https://huggingface.co/black-forest-labs/FLUX.1-dev

In [ ]:
# API-anrop.
# Skapa en bild utgående från en text. Ändra "A painting from 1520" om annat motiv önskas.
# Eller kör ett par gånger - blir olika bilder med samma text. Kan ta ett par minuter
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
headers = {"Authorization": "Bearer " + HF_KEY}
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.content
image_bytes = query({
	"inputs": "A painting from 1520",
})
# You can access the image with PIL.Image for example
import io
from PIL import Image
image = Image.open(io.BytesIO(image_bytes))

In [ ]:
# Visa den skapade bilden.
plt.imshow(image)
plt.show()

---
#IMAGE-TO-TEXT
---
https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct

In [ ]:
# Bild att beskriva. Byt gärna ut följande länk med något från nätet.
# Högerklicka på en bild, välj 'kopiera bildadress' och ersätt länken.
image_url = "https://www.download.gubboit.se/hooks.JPG"
get_img_to_file(image_url, "image")
plt.imshow(plt.imread('image'))

In [ ]:
# API-anrop.
# Beskriv med text innehållet i en bild
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-11B-Vision-Instruct"
from huggingface_hub import InferenceClient
client = InferenceClient(api_key=HF_KEY)
for message in client.chat_completion(
	model="meta-llama/Llama-3.2-11B-Vision-Instruct",
	messages=[
		{
			"role": "user",
			"content": [
				{"type": "image_url", "image_url": {"url": image_url}},
				{"type": "text", "text": "Describe this image in one sentence."},
			],
		}
	],
	max_tokens=500,
	stream=True,
):
	print(message.choices[0].delta.content, end="")

---
#OBJECT DETECTION
---
https://huggingface.co/facebook/detr-resnet-50

Verkar inte känna igen träd eller hus mm men är bra på bilar.

In [ ]:
# Funktion för att plotta bounding boxes till objekten
def plot_objects(content_path, output, score_min):
    # output är ett dictionary {}
    scores = [item['score'] for item in output]
    classes = [item['label'] for item in output]
    boxes = [item['box'] for item in output]

    for i in range(len(scores)):
        # skriv ut om score >= score_min
        if (scores[i] >= score_min):
            print(str(i), classes[i], scores[i])

    # plotta bilden med bounding boxes och objekt-nummer
    edge = ['r', 'y', 'b', 'g']
    image = Image.open(content_path)
    # Set the size of the plot figure
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.axis('off')
    # Get the current reference
    ax = plt.gca()

    for i in range(len(scores)):
        if (scores[i] >= score_min):
          tmp = boxes[i]
          ymin = tmp['ymin']
          ymax = tmp['ymax']
          xmin = tmp['xmin']
          xmax = tmp['xmax']

          # Create a Rectangle patch
          #width = image.width*(xmax - xmin)
          #height = image.height*(ymax - ymin)
          width = (xmax - xmin)
          height = (ymax - ymin)
          rect = Rectangle((xmin, ymin), width, height,
                linewidth=2, edgecolor=edge[i%4], facecolor='none')
          plt.text(xmin, ymin, str(i), color=edge[i%4],
                  fontdict={"fontsize":10,"fontweight":'bold',"ha":"left", "va":"baseline"})
          # Add the patch to the Axes
          ax.add_patch(rect)
    plt.show()

In [ ]:
# Bilder för object detection.
# Byt gärna ut följande länk med något från nätet.
# Högerklicka på en bild, välj 'kopiera bildadress' och ersätt länken.
image_url = "https://live.staticflickr.com/65535/53055525000_d57ca47b9c_b.jpg"
get_img_to_file(image_url, "image1")
image_url = "https://www.download.gubboit.se/hooks.JPG"
get_img_to_file(image_url, "image2")
plt.imshow(plt.imread('image1'))

In [ ]:
plt.imshow(plt.imread('image2'))

In [ ]:
# API-anrop. Object Detection.
API_URL = "https://api-inference.huggingface.co/models/facebook/detr-resnet-50"
headers = {"Authorization": "Bearer " + HF_KEY}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output1 = query("image1")
output2 = query("image2")

In [ ]:
# Plotta resultatet
plot_objects("image1", output1, 0.5)

In [ ]:
plot_objects("image2", output2, 0.1)

---
#Bildsegmentering
---
För **object detection** markeras varje objekt med en rektangel. För **bildsegmentering** markeras objektet på pixel-nivå. Vi ser på så vis objektets form.

https://huggingface.co/nvidia/segformer-b0-finetuned-ade-512-512

In [ ]:
# Bild för segmenering.
# Byt gärna ut följande länk med något från nätet.
# Högerklicka på en bild, välj 'kopiera bildadress' och ersätt länken.
get_img_to_file("https://www.download.gubboit.se/jakan.JPG", "image")

In [ ]:
#API-anrop: Bildsegmentering
API_URL = "https://api-inference.huggingface.co/models/nvidia/segformer-b0-finetuned-ade-512-512"
headers = {"Authorization": "Bearer " + HF_KEY}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query("image")

In [ ]:
# 'mask' är ett bildsegment
output[0]

In [ ]:
# lista med hittade objekt: nr, label
for i in range(len(output)):
  tmp = output[i]
  print(i, tmp['label'])

In [ ]:
# Visa bild för segementering
plt.imshow(plt.imread('image'))


In [ ]:
# Visa mask för ett objekt
import base64
import io
# Du kan ändra objekt-nummer enligt listan ovan
tmp = output[7]
imgdata = base64.b64decode(tmp['mask'])
# Use BytesIO to treat the byte string as an in-memory file
image = Image.open(io.BytesIO(imgdata))
# Convert the PIL Image to a NumPy array
image_np = np.array(image)
plt.imshow(image_np, cmap='gray')
plt.show()

In [ ]:
# Hämta bild och visa storlek
image = plt.imread('image')
image.shape

In [ ]:
# Visa storlek av mask-bild
mask = image_np
mask.shape

In [ ]:
# Applicera masken på vår bild.
# I masken har pixlarna värdet 255 annars 0
# pixlar som finns utanför masken blir vita
# puxlar som finns inom maske ändras inte
masked_image = image.copy()
for i in range(mask.shape[0]):
  for j in range(mask.shape[1]):
    if mask[i,j] == 0:
      masked_image[i,j,0] = 255
      masked_image[i,j,1] = 255
      masked_image[i,j,2] = 255
plt.imshow(masked_image)

---
#ASR - Automatic Speech Recognition d.v.s. tal till text.
---
https://huggingface.co/openai/whisper-large-v3

In [ ]:
# Hämta ljudfil
get_img_to_file("https://www.download.gubboit.se/harvard.wav", "harvard.wav")
get_img_to_file("https://www.download.gubboit.se/jackhammer.wav", "jackhammer.wav")
get_img_to_file("https://www.download.gubboit.se/Media1.wav", "Media1.wav")


In [ ]:
#API-anrop: ASR
API_URL = "https://api-inference.huggingface.co/models/openai/whisper-large-v3"
headers = {"Authorization": "Bearer " + HF_KEY}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

print(query("harvard.wav"))
print(query("jackhammer.wav"))
print(query("Media1.wav"))

In [ ]:
# Lyssna på ljudfiler
from IPython.display import Audio, display
display(Audio("harvard.wav", autoplay=True))


In [ ]:
display(Audio("jackhammer.wav", autoplay=True))

In [ ]:
display(Audio("Media1.wav", autoplay=True))